In [72]:
# Notes
# LLM (GPT4) is not good at identifying columns names if provided a SQL query (Test conducted using 100 quesry sample and success rate was 44%)

#import libraries
import pandas as pd
#import snowflake.connector
import numpy as np
import openai
import os
import json

# Read openai api key from file apikey.txt
api_key_file = 'apikey.txt'
if os.path.isfile(api_key_file):
    with open(api_key_file) as f:
        openai.api_key = f.readline()
else:
    print(f"Error: {api_key_file} not found.")

OPENAI_API_KEY = openai.api_key
content = ""

#login to snowflake db
"""con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='BOOTCAMP_WH',
                                 database='SCALE_PLAYPEN',
                                 role='SNOWFLAKE_USERS')
cs = con.cursor()
"""

"con = snowflake.connector.connect(user='vishal.kumar@scale.com',\n                                 account='pxa65918',\n                                 authenticator='externalbrowser',\n                                 warehouse='BOOTCAMP_WH',\n                                 database='SCALE_PLAYPEN',\n                                 role='SNOWFLAKE_USERS')\ncs = con.cursor()\n"

In [73]:
#List various prompt components
Prompt_Beginning = "You are an expert "
Prompt_Role = ""
Prompt_Mid = ". You are tasked with asking relevant questions about various assets, trade positions and holdings of the business. The questions you ask should be variations of following question:\n "
Prompt_Human_Question = ""
Prompt_End = "\nUsing the above information, ask a new question about the business. Do not include database terminology. Do not include direct columnn names. Ask question in natural business language. Only respond with question and nothing else, no explanation or other information."
Prompt_Column_Name = ""
Prompt_Column_Definition = ""
Prompt_Complex_Mid = ". You are tasked with asking relevant questions about various assets, trade positions and holdings of the business. Some examples of the questions you can ask are listed below\n "
Prompt_Complex_MidEnd = "You will now be provided with a few identifier-columns with their definitions and one value-column. Use one or more of identifier columns to build various types of aggregations of the value-column. You can use these aggregations: SUM, AVERAGE, COUNT, MAX, MIN, STANDARD DEVIATION, VARIANCE etc. You can also include as-of-date or day/time in the question\n"
Prompt_Temperature = 0.0

In [74]:
#Save customer provided questions in a dataframe
qdf = pd.read_csv('Customer_Questions.csv')
qdf = qdf.dropna()
qdf = qdf.reset_index(drop=True)
#print(qdf.head())

#Save all the used column names with their definitions in a dataframe
cdf = pd.read_csv('Column_Definitions.csv')
cdf = cdf.dropna()
cdf = cdf.reset_index(drop=True)
print(cdf.head())

#Save various roles in a list called roles
roles = ['Financial Analyst', 'Portfolio Manager', 'Risk Manager', 'Trader', 'Executive']

                      Column_Name                 Column_Definition  \
0                core_level1_code   Asset Type Level 1 - Identifier   
1       stat_fx_book_value_amount        Book Value - Dollar Amount   
2                 risk_owner_name           Risk Owner - Identifier   
3           reinsurance_deal_code     Reinsurance Deal - Identifier   
4  security_instrument_identifier  Security Instrument - Identifier   

       Used as  
0   Identifier  
1  Aggregation  
2   Identifier  
3   Identifier  
4   Identifier  


In [75]:
Prompt_Role = np.random.choice(roles)
Prompt_Human_Question_1 = np.random.choice(qdf['Question'])
Prompt_Human_Question_2 = np.random.choice(qdf['Question'])
Prompt_Human_Question_3 = np.random.choice(qdf['Question'])
# Assign a random value between 0.1 and 0.4 to prompt_temperature variable
Prompt_Temperature = np.random.uniform(0.1, 0.4)
Identifier_Column_Name_1 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
Identifier_Column_Definition_1 = cdf[cdf['Column_Name'] == Identifier_Column_Name_1]['Column_Definition']
Identifier_Column_Name_2 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
if Identifier_Column_Name_1 == Identifier_Column_Name_2:
    Identifier_Column_Name_2 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
Identifier_Column_Definition_2 = cdf[cdf['Column_Name'] == Identifier_Column_Name_2]['Column_Definition']
Identifier_Column_Name_3 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
if Identifier_Column_Name_1 == Identifier_Column_Name_3 or Identifier_Column_Name_2 == Identifier_Column_Name_3:
    Identifier_Column_Name_3 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
Identifier_Column_Definition_3 = cdf[cdf['Column_Name'] == Identifier_Column_Name_3]['Column_Definition']
Aggregation_Column_Name = np.random.choice(cdf[cdf['Used as'] == 'Aggregation']['Column_Name'])
Aggregation_Column_Definition = cdf[cdf['Column_Name'] == Aggregation_Column_Name]['Column_Definition']

In [86]:
#Create all necessary functions

#make gpt calls
def question_gpt(prompt):
    completion = openai.ChatCompletion.create(
        model="gpt-4-1106-preview",
        messages=[{"role":"user", "content":prompt}],
        temperature=Prompt_Temperature
    )
    # Parse the response to get only the "content" part and save it in the response variable
    response = completion["choices"][0]["message"]
    return response


#combine various prompt elements for a basic prompt
def prompt_compiler_basic():
    #Choose a random role from list of roles and save it in variable Prompt_Role
    Prompt_Role = np.random.choice(roles)
    #Choose a random question from the list of questions in column called 'Question' in qdf dataframe and save it in variable Prompt_Human_Question
    Prompt_Human_Question = np.random.choice(qdf['Question'])
    #Assign a random value between 0.4 and 0.8 to prompt_temperture variable
    Prompt_Temperture = np.random.uniform(0.4,0.8)
    prompt = Prompt_Beginning + Prompt_Role + Prompt_Mid + Prompt_Human_Question + Prompt_End
    return prompt

#combine various prompt elements for a complex prompt
def prompt_compiler_complex():
    Prompt_Role = np.random.choice(roles)
    Prompt_Human_Question_1 = np.random.choice(qdf['Question'])
    Prompt_Human_Question_2 = np.random.choice(qdf['Question'])
    Prompt_Human_Question_3 = np.random.choice(qdf['Question'])
    # Assign a random value between 0.1 and 0.4 to prompt_temperature variable
    Prompt_Temperature = np.random.uniform(0.1, 0.4)
    Identifier_Column_Name_1 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
    Identifier_Column_Definition_1 = cdf.loc[cdf['Column_Name'] == Identifier_Column_Name_1, 'Column_Definition'].values[0]
    Identifier_Column_Name_2 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
    if Identifier_Column_Name_1 == Identifier_Column_Name_2:
        Identifier_Column_Name_2 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
    Identifier_Column_Definition_2 = cdf.loc[cdf['Column_Name'] == Identifier_Column_Name_2, 'Column_Definition'].values[0]
    Identifier_Column_Name_3 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
    if Identifier_Column_Name_1 == Identifier_Column_Name_3 or Identifier_Column_Name_2 == Identifier_Column_Name_3:
        Identifier_Column_Name_3 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
    Identifier_Column_Definition_3 = cdf.loc[cdf['Column_Name'] == Identifier_Column_Name_3, 'Column_Definition'].values[0]
    Aggregation_Column_Name = np.random.choice(cdf[cdf['Used as'] == 'Aggregation']['Column_Name'])
    Aggregation_Column_Definition = cdf.loc[cdf['Column_Name'] == Aggregation_Column_Name, 'Column_Definition'].values[0]
    prompt = Prompt_Beginning + Prompt_Role + Prompt_Complex_Mid + "\n" + Prompt_Human_Question_1 + "\n" + Prompt_Human_Question_2 + "\n" + Prompt_Human_Question_3 + "\n\n" + Prompt_Complex_MidEnd + "\nIdentifier-Column Name:" + Identifier_Column_Name_1 + " Identifier-Column Definition:" + Identifier_Column_Definition_1 + "\nIdentifier-Column Name:" + Identifier_Column_Name_2 + " Identifier-Column Definition:" + Identifier_Column_Definition_2 + "\nIdentifier-Column Name:" + Identifier_Column_Name_3 + " Identifier-Column Definition:" + Identifier_Column_Definition_3 + "\nAggregation-Column Name:" + Aggregation_Column_Name + " Aggregation-Column Definition:" + Aggregation_Column_Definition +  "\n" + Prompt_End
    return prompt

#create function prompt_cot which takes conversation_history as input and returns a response
def prompt_cot(conversation_history):
    #compile conversation history
    sc_prompt = "Read our conversation history provided below and answer the following questions:\n" + conversation_history + "\n1. Did you provide the best response? If yes, then respond 'Yes'. \nIf no, provide the improved response question and improved response only, nothing else, no apologies needed. Dont say No in response."
    sc_response = question_gpt(sc_prompt)
    complexity_prompt = "Read our conversation history provided below and answer the following questions:\n" + conversation_history + "\n2. Was the question asked by you complex enough? If yes, then respond 'Yes'. \nIf no, provide a question with higher complexity only, nothing else, no apologies needed."
    complexity_response = question_gpt(complexity_prompt)
    return sc_response, complexity_response


In [87]:
#call prompt_compiler_complex() function to generate a prompt
prompt = prompt_compiler_complex()
#call question_gpt() function to generate a response
response = question_gpt(prompt)
#save json key content from response in response
response = response['content']
#print(response)
conversation_history = "My question to you:\n" + prompt + "\n\n\nYour response to my question:" + response
print(conversation_history)
#call prompt_cot() function to check self-consistency and complexity
sc_response, complexity_response = prompt_cot(conversation_history)
print(sc_response)
print(complexity_response)


My question to you:
You are an expert Financial Analyst. You are tasked with asking relevant questions about various assets, trade positions and holdings of the business. Some examples of the questions you can ask are listed below
 
how many distinct assets are rated by Egan Jones?
How has the average book yield changed YTD by Asset class?
Which deals have the best performing yield?

You will now be provided with a few identifier-columns with their definitions and one value-column. Use one or more of identifier columns to build various types of aggregations of the value-column. You can use these aggregations: SUM, AVERAGE, COUNT, MAX, MIN, STANDARD DEVIATION, VARIANCE etc. You can also include as-of-date or day/time in the question

Identifier-Column Name:security_instrument_identifier Identifier-Column Definition:Security Instrument - Identifier
Identifier-Column Name:portfolio_identifier Identifier-Column Definition:Portfolio - Identifier
Identifier-Column Name:naic_designation_code 